## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [2]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [3]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

df_integrado = pd.read_csv("../../data/processed/datos_integrados_mc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


In [4]:
df_integrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_cliente             10127 non-null  float64
 1   edad                   10127 non-null  int64  
 2   importe_solicitado     10127 non-null  int64  
 3   duracion_credito       10127 non-null  int64  
 4   antiguedad_empleado    9790 non-null   float64
 5   situacion_vivienda     10127 non-null  object 
 6   ingresos               10127 non-null  int64  
 7   objetivo_credito       10127 non-null  object 
 8   pct_ingreso            10127 non-null  float64
 9   tasa_interes           9215 non-null   float64
 10  estado_credito         10127 non-null  int64  
 11  falta_pago             10127 non-null  object 
 12  completitud_fila_x     10127 non-null  float64
 13  situacion_vivienda_ok  10127 non-null  bool   
 14  antiguedad_cliente     10127 non-null  float64
 15  es

---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [5]:
# Se establece qué columnas se eliminan

col_eliminar = ['completitud_fila_x', 'completitud_fila_x','situacion_vivienda_ok','situacion_vivienda_ok','completitud_fila_y','nivel_tarjeta']

# Se ejecuta la operación

df_integrado.drop(col_eliminar, inplace=True, axis=1)

In [6]:
print("Vista del dataset integrado:")
display(df_integrado.head(1))


Vista del dataset integrado:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,falta_pago,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,N,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0


Limpieza de los datos (filtrado a nivel de filas)

In [7]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_integrado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)
df_integrado.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,...,45.0,SOLTERO,PASIVO,992.0,M,3421.0,DESCONOCIDO,21.0,0.0,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,...,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,POSGRADO_COMPLETO,23.0,3.0,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,...,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,UNIVERSITARIO_INCOMPLETO,82.0,2.0,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,...,41.0,CASADO,ACTIVO,4369.0,F,3128.0,UNIVERSITARIO_INCOMPLETO,59.0,2.0,ok


In [8]:
# Se puede definir una función para aplicar los cálculos de la Regla 2
def regla_duracion_ingresos_credito(row):
    duracion_credito = row.duracion_credito
    pct_ingreso = row.pct_ingreso
    situacion_vivienda = row.situacion_vivienda
    
    # Duración mínima permitida
    duracion_minima = df_integrado['duracion_credito'].min()
    
    # Se verifica si la duración del crédito es la mínima permitida
    if duracion_credito == duracion_minima:
        # Se verifica si el porcentaje de ingresos excede el 60%
        if pct_ingreso > 0.6:
            # Si no es propietario de su vivienda, es un error
            if situacion_vivienda != 'PROPIA':
                return 'err'
    return 'ok'

# Se aplica la función para todos los elementos del dataset
regla_duracion_ingresos = df_integrado.apply(lambda row: regla_duracion_ingresos_credito(row), axis=1).rename("regla_duracion_ingresos")

# Se unen los resultados al dataset inicial
df_integrado = pd.concat([df_integrado, regla_duracion_ingresos], axis=1)

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla 2:\n")
display(df_integrado[df_integrado.regla_duracion_ingresos == 'err'].head())

# Se verifica la cantidad de elementos
aux_regla_2 = df_integrado[df_integrado.regla_duracion_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla 2: {aux_regla_2.shape[0]}")

Se visualizan las tuplas que no cumplen con la regla 2:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos,regla_duracion_ingresos
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,CASADO,ACTIVO,2244.0,M,19802.0,UNIVERSITARIO_INCOMPLETO,58.0,4.0,err,err
541,709186983.0,23,20000,2,1.0,ALQUILER,32900,EDUCACIÓN,0.61,16.00,...,CASADO,ACTIVO,1536.0,M,8714.0,UNIVERSITARIO_INCOMPLETO,38.0,4.0,ok,err
1646,711399408.0,24,29100,2,8.0,HIPOTECA,46000,PERSONAL,0.63,12.99,...,DIVORCIADO,PASIVO,2462.0,F,12373.0,UNIVERSITARIO_COMPLETO,40.0,2.0,ok,err
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,...,CASADO,PASIVO,2275.0,F,1684.0,UNIVERSITARIO_INCOMPLETO,45.0,3.0,err,err
3041,713962233.0,22,30000,2,1.0,ALQUILER,48000,EDUCACIÓN,0.63,18.39,...,CASADO,ACTIVO,1877.0,F,3035.0,UNIVERSITARIO_COMPLETO,37.0,3.0,ok,err


Cantidad de filas que no cumplen la regla 2: 7


In [11]:
# Se filtran las filas con algún error detectado en edad
print(f"Filas antes del filtro: {df_integrado.shape[0]}")

temp = df_integrado[df_integrado['edad'] < 90]

# Otro filtro posible: por la regla de negocio agregada

temp_c = temp[temp['regla_pct_ingresos'] == 'ok']

# Se filtran las filas por la regla 2

temp_a = temp_c[temp_c['regla_duracion_ingresos'] == 'ok']

# Se filtran las filas por nulos "antiguedad_cliente"

temp_b = temp_a[temp_a['antiguedad_empleado'].notnull()]

# Se filtran las filas con algún error detectado "antiguedad_cliente"

temp_d = temp_a[temp_a['antiguedad_empleado'] < 50]

# Se filtran las filas por nulos "tasa_interes"

df_integrado = temp_d[temp_d['tasa_interes'].notnull()]

print(f"Filas después del filtro: {df_integrado.shape[0]}")

Filas antes del filtro: 8878
Filas después del filtro: 8878


Se eliminan columnas de las reglas

In [12]:
df_integrado.drop('regla_pct_ingresos', inplace=True, axis=1)
df_integrado.drop('regla_duracion_ingresos', inplace=True, axis=1)

Integración de datos

El dataset ya se encuentra integrado con el nombre de df_integrado

In [13]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 21


#### Transformación de atributos

Atributos nominales que se modifican los valores

In [14]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')

# Columna: estado_credito
cambios_estado_credito = {
    0: 'P',
    1 : 'C',
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')



# Sobre este resultado será necesario eliminar las columnas auxiliares

Atributos numéricos que se discretizan

In [15]:
df_integrado['operaciones_ult_12m'].describe()

count    8878.000000
mean       65.018135
std        23.517037
min        10.000000
25%        45.000000
50%        67.000000
75%        81.000000
max       139.000000
Name: operaciones_ult_12m, dtype: float64

In [16]:
# Antiguedad del empleado

etiquetas_a_e = ['menor_5', '5_a_10', 'mayor_10']
rangos_a_e = [0, 4, 10, 50]
valor_para_nan = 'NA'
antiguedad_empleados_N = pd.cut(df_integrado['antiguedad_empleado'], 
                                bins=rangos_a_e, 
                                labels=etiquetas_a_e,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

antiguedad_empleados_N.value_counts()

# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_integrado['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

edad_N.value_counts()

# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_integrado['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

pct_ingreso_N.value_counts()

# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_integrado['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

ingresos_N.value_counts()

# tasa_interes

etiquetas_t_i = ['hasta_7p', '7p_a_15p', '15p_a_20p', 'mayor_20p']
rangos_t_i = [0, 6.99, 14.99, 19.99, 100]
tasa_interes_N = pd.cut(df_integrado['tasa_interes'], 
                                bins=rangos_t_i, 
                                labels=etiquetas_t_i)

tasa_interes_N.value_counts()

# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_integrado['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

antiguedad_cliente_N.value_counts()

# limite_credito_tc

etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
rangos_l_tc = [0, 2999, 4999, 9999, 100000]
limite_credito_tc_N = pd.cut(df_integrado['limite_credito_tc'], 
                                bins=rangos_l_tc, 
                                labels=etiquetas_l_tc)

limite_credito_tc_N.value_counts()

# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '2k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_integrado['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()

# operaciones_ult_12m

etiquetas_o_u12 = ['menor_15', '15_a_30', '30_a_50', '50_a_75', '75_a_100', 'mayor_100']
rangos_o_u12 = [0, 14, 29, 49, 74, 99, 1000]
operaciones_ult_12m_N = pd.cut(df_integrado['operaciones_ult_12m'], 
                                bins=rangos_o_u12, 
                                labels=etiquetas_o_u12)

operaciones_ult_12m_N.value_counts()

operaciones_ult_12m
50_a_75      2984
75_a_100     2653
30_a_50      2091
mayor_100     608
15_a_30       520
menor_15       22
Name: count, dtype: int64

In [17]:
df_integrado.columns

Index(['id_cliente', 'edad', 'importe_solicitado', 'duracion_credito',
       'antiguedad_empleado', 'situacion_vivienda', 'ingresos',
       'objetivo_credito', 'pct_ingreso', 'tasa_interes', 'estado_credito',
       'falta_pago', 'antiguedad_cliente', 'estado_civil', 'estado_cliente',
       'gastos_ult_12m', 'genero', 'limite_credito_tc', 'nivel_educativo',
       'operaciones_ult_12m', 'personas_a_cargo'],
      dtype='object')

In [18]:
col_eliminar_final = [
              'edad',
              'antiguedad_empleado',
              'antiguedad_cliente',
              'estado_civil',
              'estado_credito',
              'ingresos',
              'pct_ingreso', 
              'tasa_interes',
              'gastos_ult_12m', 
              'limite_credito_tc', 
              'operaciones_ult_12m',
              'id_cliente']

df_integrado.drop(col_eliminar_final, inplace=True, axis=1)


df_final = pd.concat([estado_civil_N, estado_credito_N, operaciones_ult_12m_N, gastos_ult_12m_N, limite_credito_tc_N, antiguedad_cliente_N, tasa_interes_N, ingresos_N, pct_ingreso_N, antiguedad_empleados_N, edad_N, df_integrado], axis=1)
df_final.head(5)


,estado_civil_N,estado_credito_N,operaciones_ult_12m,gastos_ult_12m,limite_credito_tc,antiguedad_cliente,tasa_interes,ingresos,pct_ingreso,antiguedad_empleado,edad,importe_solicitado,duracion_credito,situacion_vivienda,objetivo_credito,falta_pago,estado_cliente,genero,nivel_educativo,personas_a_cargo
0,C,P,mayor_100,mayor_10k,3k_a_5k,2y_a_4y,hasta_7p,50k_a_100k,hasta_20,5_a_10,menor_25,11000,3,HIPOTECA,INVERSIONES,N,ACTIVO,F,SECUNDARIO_COMPLETO,3.0
1,S,C,15_a_30,menor_1k,3k_a_5k,2y_a_4y,15p_a_20p,20k_a_50k,hasta_20,menor_5,menor_25,1500,2,ALQUILER,MEJORAS_HOGAR,Y,PASIVO,M,DESCONOCIDO,0.0
2,D,P,15_a_30,2k_a_4k,5k_a_10k,2y_a_4y,7p_a_15p,20k_a_50k,20_a_40,5_a_10,menor_25,10000,2,OTROS,EDUCACIÓN,Y,PASIVO,M,POSGRADO_COMPLETO,3.0
3,S,P,75_a_100,2k_a_4k,menor_3k,menor_2y,7p_a_15p,20k_a_50k,20_a_40,menor_5,25_a_30,6000,4,ALQUILER,INVERSIONES,N,ACTIVO,F,UNIVERSITARIO_INCOMPLETO,2.0
4,C,P,50_a_75,4k_a_6k,3k_a_5k,2y_a_4y,7p_a_15p,50k_a_100k,20_a_40,menor_5,25_a_30,10000,2,HIPOTECA,EDUCACIÓN,N,ACTIVO,F,UNIVERSITARIO_INCOMPLETO,2.0


In [19]:
print(df_final.shape[0])
print(df_final.shape[1])

8878
20


Exportación de metadatos

In [20]:
import dtale as dt

informe = dt.show(df_final)
informe.open_browser()

c:\Users\Paula Domski\Documents\Master_BigData\MetodologiaGestionProyectos\SesionPractica1\13MBID-OCT2324\.venv\lib\site-packages\dtale\column_analysis.py:402: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

c:\Users\Paula Domski\Documents\Master_BigData\MetodologiaGestionProyectos\SesionPractica1\13MBID-OCT2324\.venv\lib\site-packages\dtale\column_analysis.py:402: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

c:\Users\Paula Domski\Documents\Master_BigData\MetodologiaGestionProyectos\SesionPractica1\13MBID-OCT2324\.venv\lib\site-packages\dtale\column_analysis.py:402: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

2024-05-06 17:56:27,630 - ERROR    - Exception occurred while processing request: Cannot setitem on a Categorical w

Exportación del dataset

In [38]:
df_final.to_csv("../../data/final/datos_finales.csv", sep=';', index=False)